In [13]:
# create_test_split.py
import pandas as pd
import argparse
import os

def split_dataset(file_path = "post_bt.csv", output_dir="bt_split/", val_ratio=0.05, seed=42):
    df = pd.read_csv(file_path)
    n = len(df)
    half = n // 2

    df_orig = df.iloc[:half].reset_index(drop=True)
    df_aug = df.iloc[half:].reset_index(drop=True)

    print(f"✅ Loaded dataset: {n} samples")
    print(f"   Original: {len(df_orig)}, Augmented: {len(df_aug)}")

    def stratified_split(df, val_ratio):
        """Split a single source (orig or aug) into train/val/test."""
        n_total = len(df)
        n_val = int(n_total * val_ratio)

        df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
        df_val = df_shuffled.iloc[:n_val]
        df_train = df_shuffled.iloc[n_val:]

        return df_train, df_val

    # Split original and augmented separately
    orig_train, orig_val = stratified_split(df_orig, val_ratio)
    aug_train, aug_val = stratified_split(df_aug, val_ratio)

    # Combine them (balanced parts)
    df_train = pd.concat([orig_train, aug_train]).sample(frac=1, random_state=seed).reset_index(drop=True)
    df_val = pd.concat([orig_val, aug_val]).sample(frac=1, random_state=seed).reset_index(drop=True)

    # Make output dir
    os.makedirs(output_dir, exist_ok=True)

    # Save all splits
    df_train.to_csv(os.path.join(output_dir, "train.csv"), index=False)
    df_val.to_csv(os.path.join(output_dir, "val.csv"), index=False)

    print(f"✅ Saved splits to {output_dir}")
    print(f"Train: {len(df_train)} | Val: {len(df_val)}")
split_dataset()

✅ Loaded dataset: 8802 samples
   Original: 4401, Augmented: 4401
✅ Saved splits to bt_split/
Train: 8362 | Val: 440
